In [1]:
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import requests as req
import time
import pandas as pd

# NASA NEWS

In [2]:
#set up splinter 
executable_path = {'executable_path':'/temp/chromedriver'}
browser = Browser('chrome', **executable_path)

#visit site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

#parsing html
html_code = browser.html
soup = BeautifulSoup(html_code, 'html.parser')

#collect titles and paragraphs
news_title = soup.find('div', class_='content_title').text
paragraph = soup.find('div', class_='article_teaser_body').text

browser.quit()

In [3]:
print(news_title)
print(paragraph)

After a Reset, Curiosity Is Operating Normally
NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


# NASA JPL Mars Space Images - Featured Image

In [17]:
# launch browser
executable_path = {'executable_path':'/temp/chromedriver'}
browser = Browser('chrome', **executable_path)
# Visit the url for JPL Featured Space Image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
html_code = browser.html
soup = BeautifulSoup(html_code, 'html.parser')

In [18]:
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

In [20]:
browser.click_link_by_partial_text('more info')

In [22]:
#get image html code
image_html = browser.html
image_soup = BeautifulSoup(image_html, 'html.parser')

image_path = image_soup.find('figure', class_='lede').a['href']
featured_image_url = 'https://www.jpl.nasa.gov' + image_path

In [23]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16567_hires.jpg


# Mars Weather

In [9]:
#getting weather on mars from twitter feed
twitter_mars_weather = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_mars_weather)
weather_html = browser.html
twitter_soup = BeautifulSoup(weather_html, 'html.parser')

In [10]:
mars_weather = twitter_soup.find_all('div', class_='js-tweet-text-container')

In [36]:
browser.quit()
print(mars_weather[3].text)


InSight sol 83 (2019-02-19) low -95.0ºC (-139.0ºF) high -11.7ºC (11.0ºF)
winds from the SW at 4.1 m/s (9.2 mph) gusting to 10.5 m/s (23.6 mph)
pressure at 7.20 hPa



# Mars Facts

In [12]:
# read page
facts_url = 'http://space-facts.com/mars/'
read_mars_facts = pd.read_html(facts_url)
#create dataframe
mars_df = read_mars_facts[0]
mars_df.columns = ['Description','Value']
mars_df.set_index('Description', inplace=True)

# display df
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [13]:
#convert to html string.
mars_html = mars_df.to_html()
mars_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

# Hemispheres

In [14]:
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
usgs_req = req.get(usgs_url)
hemi_soup = BeautifulSoup(usgs_req.text, 'html.parser')
hemi_list = hemi_soup.find_all('a', class_='itemLink product-item')

In [15]:
hemisphere_image_urls = []
for hemi_img in hemi_list:
    img_title = hemi_img.find('h3').get_text().replace('Enhanced', '')
    hemi_img_link = 'https://astrogeology.usgs.gov/' + hemi_img['href']
    img_soup = BeautifulSoup(req.get(hemi_img_link).text, 'html.parser')
    img_download = img_soup.find('div', {'class':'downloads'})
    img_url = img_download.find('a')['href']
    hemisphere_image_urls.append({'title': img_title, 'img_url': img_url})

In [16]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]